In [129]:
import pandas as pd
import os

In [130]:
loc = pd.read_csv("../data/locations.csv", dtype="str")
loc.head()

,location_code,cidade,bairro,logradouro,location,neighborhood_code,city_code
0,12658,limeira,vila claudia,avenida antonio ometo,limeira|vila claudia|avenida antonio ometo,5215,202
1,1427,birigui,chacaras caicara,rua benjamin strozi,birigui|chacaras caicara|rua benjamin strozi,683,43
2,1440,birigui,jardim vitrine,rua silvio vieira coelho,birigui|jardim vitrine|rua silvio vieira coelho,2837,43
3,17021,praia grande,solemar ii,avenida ministro marcos freire,praia grande|solemar ii|avenida ministro marco...,4848,305
4,11939,jardinopolis,jardinopolis,avenida prefeito newton reis,jardinopolis|jardinopolis|avenida prefeito new...,2858,187


In [131]:
loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37545 entries, 0 to 37544
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   location_code      37545 non-null  object
 1   cidade             37545 non-null  object
 2   bairro             37545 non-null  object
 3   logradouro         37545 non-null  object
 4   location           37545 non-null  object
 5   neighborhood_code  37545 non-null  object
 6   city_code          37545 non-null  object
dtypes: object(7)
memory usage: 2.0+ MB


In [132]:
files = os.listdir("ceps")

ceps = pd.concat([
    pd.read_csv(f"ceps/{f}", names=["cep", "logradouro", "complemento", "bairro", "id_cidade", "id_estado"], index_col=False, dtype="str") 
    for f in files
])
ceps.head()

,cep,logradouro,complemento,bairro,id_cidade,id_estado
0,01001000,Praça da Sé,- lado ímpar,Sé,8966,26
1,01001001,Praça da Sé,- lado par,Sé,8966,26
2,01001010,Rua Filipe de Oliveira,NaN,Sé,8966,26
3,01001900,Praça da Sé 108,NaN,Sé,8966,26
4,01001901,Praça da Sé 371,NaN,Sé,8966,26


In [133]:
ceps.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 301635 entries, 0 to 60326
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   cep          301635 non-null  object
 1   logradouro   300946 non-null  object
 2   complemento  41242 non-null   object
 3   bairro       300852 non-null  object
 4   id_cidade    301635 non-null  object
 5   id_estado    301635 non-null  object
dtypes: object(6)
memory usage: 16.1+ MB


In [134]:
ceps["logradouro"] = ceps["logradouro"].str.normalize("NFKD").str.encode("ascii",  errors='ignore').str.decode("UTF-8").str.lower()

In [135]:
ceps.drop_duplicates(subset=["logradouro"], keep="first", inplace=True)
ceps.dropna(subset=["logradouro"], inplace=True)
ceps = ceps.reset_index()

In [136]:
ceps[ceps["logradouro"].str.contains("praca da se")]

,index,cep,logradouro,complemento,bairro,id_cidade,id_estado
0,0,01001000,praca da se,- lado ímpar,Sé,8966,26
2,3,01001900,praca da se 108,NaN,Sé,8966,26
3,4,01001901,praca da se 371,NaN,Sé,8966,26
4,5,01001902,praca da se 385,NaN,Sé,8966,26


In [137]:
loc[loc["logradouro"].str.contains("praca da se")]

,location_code,cidade,bairro,logradouro,location,neighborhood_code,city_code
15127,32192,s.paulo,se,"praca da se, 371","s.paulo|se|praca da se, 371",4788,358


In [138]:
new_loc = loc.join(ceps.set_index("logradouro"), on="logradouro", rsuffix="_cep", how="left")

In [139]:
new_loc = new_loc[[*loc.columns, "cep"]]
new_loc.head()

,location_code,cidade,bairro,logradouro,location,neighborhood_code,city_code,cep
0,12658,limeira,vila claudia,avenida antonio ometo,limeira|vila claudia|avenida antonio ometo,5215,202,NaN
1,1427,birigui,chacaras caicara,rua benjamin strozi,birigui|chacaras caicara|rua benjamin strozi,683,43,NaN
2,1440,birigui,jardim vitrine,rua silvio vieira coelho,birigui|jardim vitrine|rua silvio vieira coelho,2837,43,16200073
3,17021,praia grande,solemar ii,avenida ministro marcos freire,praia grande|solemar ii|avenida ministro marco...,4848,305,11717260
4,11939,jardinopolis,jardinopolis,avenida prefeito newton reis,jardinopolis|jardinopolis|avenida prefeito new...,2858,187,NaN


In [140]:
new_loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37545 entries, 0 to 37544
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   location_code      37545 non-null  object
 1   cidade             37545 non-null  object
 2   bairro             37545 non-null  object
 3   logradouro         37545 non-null  object
 4   location           37545 non-null  object
 5   neighborhood_code  37545 non-null  object
 6   city_code          37545 non-null  object
 7   cep                29482 non-null  object
dtypes: object(8)
memory usage: 2.3+ MB


In [141]:
new_loc.to_csv("../data/locations2.csv", index=False)